# Data Cleaning and Combining
Remove unnecessary, useless data and combine the two datasets into one dataset

Importing libraries & files

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [9]:
calfires = gpd.read_file("fire22_1.gdb")
calfires

,YEAR_,STATE,AGENCY,UNIT_ID,TREATMENT_ID,TREATMENT_NAME,START_DATE,END_DATE,TREATED_AC,GIS_ACRES,RX_CONSUM,PRE_CON_CLASS,POST_CON_CLASS,TREATMENT_TYPE,Shape_Length,Shape_Area,geometry
0,2020,CA,CDF,AEU,10509,Jan 31 2020 Broadcast,2020-01-31 00:00:00+00:00,2020-01-31T00:00:00+00:00,15.000000,15.047042,NaN,NaN,NaN,1.0,1365.887529,6.089322e+04,"MULTIPOLYGON (((-45261.902 81865.442, -45248.3..."
1,2020,CA,PVT,AEU,10572,2020 Fuels Reduction,2020-02-13 00:00:00+00:00,2020-02-13T00:00:00+00:00,22.300000,36.739700,NaN,NaN,NaN,1.0,2681.610336,1.486803e+05,"MULTIPOLYGON (((-57222.192 98500.043, -57231.6..."
2,2020,CA,CDF,AEU,10630,Feb 2020 Broadcast,2020-02-13 00:00:00+00:00,2020-02-18T00:00:00+00:00,22.400000,38.839832,NaN,NaN,NaN,1.0,3918.219956,1.571792e+05,"MULTIPOLYGON (((-49821.237 54143.575, -49815.4..."
3,2020,CA,CDF,AEU,10639,Feb 2020 Broadcast,2020-02-11 00:00:00+00:00,2020-02-20T00:00:00+00:00,75.500000,75.498100,NaN,NaN,NaN,1.0,3918.834462,3.055300e+05,"MULTIPOLYGON (((-47052.539 80922.773, -47052.5..."
4,2020,CA,CDF,AEU,10780,Mar 3 2020 Broadcast,2020-03-03 00:00:00+00:00,2020-03-03T00:00:00+00:00,61.800000,61.771519,NaN,NaN,NaN,1.0,2769.034316,2.499805e+05,"MULTIPOLYGON (((-47591.189 80696.956, -47590.2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8630,2022,CA,CSP,119,049,Broadcast Burn,2022-09-26 00:00:00+00:00,2022-11-02T00:00:00+00:00,106.348724,106.348297,NaN,NaN,NaN,1.0,8293.818091,4.303780e+05,"MULTIPOLYGON (((-339653.068 267775.462, -33965..."
8631,2022,CA,CSP,956,048,Broadcast Burn,2022-09-26 00:00:00+00:00,2022-09-27T00:00:00+00:00,97.264578,97.264191,NaN,NaN,NaN,1.0,3331.356950,3.936158e+05,"MULTIPOLYGON (((-338700.258 267705.477, -33870..."
8632,2022,CA,CSP,119,043,Broadcast Burn,2022-10-27 00:00:00+00:00,2022-11-02T00:00:00+00:00,44.108643,44.108467,NaN,NaN,NaN,1.0,7161.380389,1.785013e+05,"MULTIPOLYGON (((-341472.748 267479.256, -34147..."
8633,2022,CA,CSP,119,040,Broadcast Burn,2022-10-27 00:00:00+00:00,2022-11-02T00:00:00+00:00,32.861754,32.861622,NaN,NaN,NaN,1.0,3955.312226,1.329868e+05,"MULTIPOLYGON (((-341942.237 267816.094, -34193..."


Removing unneeded colums and rows with missing data

In [8]:
calfires = calfires.drop(labels=["STATE", "AGENCY", "UNIT_ID", "TREATMENT_ID", "TREATMENT_NAME", "RX_CONSUM", "PRE_CON_CLASS", "POST_CON_CLASS", "TREATMENT_TYPE"], axis=1)
calfires

,YEAR_,START_DATE,END_DATE,TREATED_AC,GIS_ACRES,Shape_Length,Shape_Area,geometry
0,2020,2020-01-31 00:00:00+00:00,2020-01-31T00:00:00+00:00,15.000000,15.047042,1365.887529,6.089322e+04,"MULTIPOLYGON (((-45261.902 81865.442, -45248.3..."
1,2020,2020-02-13 00:00:00+00:00,2020-02-13T00:00:00+00:00,22.300000,36.739700,2681.610336,1.486803e+05,"MULTIPOLYGON (((-57222.192 98500.043, -57231.6..."
2,2020,2020-02-13 00:00:00+00:00,2020-02-18T00:00:00+00:00,22.400000,38.839832,3918.219956,1.571792e+05,"MULTIPOLYGON (((-49821.237 54143.575, -49815.4..."
3,2020,2020-02-11 00:00:00+00:00,2020-02-20T00:00:00+00:00,75.500000,75.498100,3918.834462,3.055300e+05,"MULTIPOLYGON (((-47052.539 80922.773, -47052.5..."
4,2020,2020-03-03 00:00:00+00:00,2020-03-03T00:00:00+00:00,61.800000,61.771519,2769.034316,2.499805e+05,"MULTIPOLYGON (((-47591.189 80696.956, -47590.2..."
...,...,...,...,...,...,...,...,...
8630,2022,2022-09-26 00:00:00+00:00,2022-11-02T00:00:00+00:00,106.348724,106.348297,8293.818091,4.303780e+05,"MULTIPOLYGON (((-339653.068 267775.462, -33965..."
8631,2022,2022-09-26 00:00:00+00:00,2022-09-27T00:00:00+00:00,97.264578,97.264191,3331.356950,3.936158e+05,"MULTIPOLYGON (((-338700.258 267705.477, -33870..."
8632,2022,2022-10-27 00:00:00+00:00,2022-11-02T00:00:00+00:00,44.108643,44.108467,7161.380389,1.785013e+05,"MULTIPOLYGON (((-341472.748 267479.256, -34147..."
8633,2022,2022-10-27 00:00:00+00:00,2022-11-02T00:00:00+00:00,32.861754,32.861622,3955.312226,1.329868e+05,"MULTIPOLYGON (((-341942.237 267816.094, -34193..."
